<a href="https://colab.research.google.com/github/rafayel404/Data-Visualization/blob/main/Plotly_objects_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import plotly.offline as pyo
import plotly.graph_objects as go

In [4]:
matches=pd.read_csv('matches.csv')
delivery=pd.read_csv('deliveries.csv')

ipl=delivery.merge(matches,left_on='match_id',right_on='id')
ipl.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN


In [5]:
# Avg vs SR graph of Top 50 batsman(in terms of total runs)

# Fetching a new dataframe with Top 50 batsman
top50=ipl.groupby('batsman')['batsman_runs'].sum().sort_values(ascending=False).head(50).index.tolist()
new_ipl=ipl[ipl['batsman'].isin(top50)]

In [9]:
# Calculating SR
# SR=[(number of runs scored)/(number of balls played)]*100
runs=new_ipl.groupby('batsman')['batsman_runs'].sum()
balls=new_ipl.groupby('batsman')['batsman_runs'].count()

sr=(runs/balls)*100

sr=sr.reset_index()
sr.head(3)

,batsman,batsman_runs
0,AB de Villiers,145.129059
1,AC Gilchrist,133.054662
2,AJ Finch,126.299213


In [8]:
# Calculating Avg
# Avg=(Total number of Runs)/(Number of outs)

# Calculating number of outs for top 50 batsman
out=ipl[ipl['player_dismissed'].isin(top50)]

nouts=out['player_dismissed'].value_counts()

avg=runs/nouts

avg=avg.reset_index()
avg.rename(columns={'index':'batsman',0:'avg'},inplace=True)

avg=avg.merge(sr,on='batsman')
avg.head(3)

,batsman,avg,batsman_runs
0,AB de Villiers,38.307692,145.129059
1,AC Gilchrist,27.223684,133.054662
2,AJ Finch,27.186441,126.299213


In [16]:
trace=go.Scatter(x=avg['avg'],y=avg['batsman_runs'],
                 mode='markers',
                 text=avg['batsman'],
                 marker={'color':'#66198c','size':15})
data=[trace]
layout=go.Layout(title='Batsman Avg VS SR',
                 xaxis={'title':'Batsman avg'},
                 yaxis={'title':'Batsman SR'}
                 )
fig=go.Figure(data=data,layout=layout)
fig.show()

### Line Chart
It's an extension of Scatter plot. Usually used to show a time series data

In [17]:
# Year by Year batsman performance

single=ipl[ipl['batsman']=='V Kohli']
performance=single.groupby('season')['batsman_runs'].sum().reset_index()
performance

,season,batsman_runs
0,2008,165
1,2009,246
2,2010,307
3,2011,557
4,2012,364
5,2013,639
6,2014,359
7,2015,505
8,2016,973
9,2017,308


In [24]:
trace=go.Scatter(x=performance['season'],y=performance['batsman_runs'],
                 mode='lines+markers',
                  marker={'color':'#66198c'}
                 )
data=[trace]
layout=go.Layout(title='Runs of Virat',
                 xaxis={'title':'Season'},
                 yaxis={'title':'Runs'}
                 )
fig=go.Figure(data=data,layout=layout)
fig.show()

In [26]:

single1=ipl[ipl['batsman']=='MS Dhoni']
performance1=single1.groupby('season')['batsman_runs'].sum().reset_index()


In [29]:
trace=go.Scatter(x=performance['season'],y=performance['batsman_runs'],
                 mode='lines+markers',
                  marker={'color':'#66198c'},
                 name='Virat'
                 )
trace1=go.Scatter(x=performance1['season'],y=performance1['batsman_runs'],
                 mode='lines+markers',
                  marker={'color':'#126e68'},
                  name='Dhoni'
                 )

data=[trace,trace1]
layout=go.Layout(title='Runs of Virat and Dhoni',
                 xaxis={'title':'Season'},
                 yaxis={'title':'Runs'}
                 )
fig=go.Figure(data=data,layout=layout)
fig.show()

In [32]:
# Multiple Line Charts

def batsman_comp(*name):
    data=[]
    for i in name:
        single=ipl[ipl['batsman']==i]
        performance=single.groupby('season')['batsman_runs'].sum().reset_index()

        trace=go.Scatter(x=performance['season'],y=performance['batsman_runs']
                         ,mode='lines + markers',name=i)

        data.append(trace)

    layout=go.Layout(title='Batsman Record Comparator',
                xaxis={'title':'Season'},
                yaxis={'title':'Runs'})

    fig=go.Figure(data=data,layout=layout)

    fig.show()


In [33]:
batsman_comp('V Kohli', 'RG Sharma','DA Warner','MS Dhoni')

### Bar

In [34]:
top10=ipl.groupby('batsman')['batsman_runs'].sum().sort_values(ascending=False).head(10).index.tolist()
top10_df=ipl[ipl['batsman'].isin(top10)]

In [35]:
top10_score=top10_df.groupby('batsman')['batsman_runs'].sum().reset_index()
top10_score

,batsman,batsman_runs
0,AB de Villiers,3486
1,CH Gayle,3651
2,DA Warner,4014
3,G Gambhir,4132
4,MS Dhoni,3560
5,RG Sharma,4207
6,RV Uthappa,3778
7,S Dhawan,3561
8,SK Raina,4548
9,V Kohli,4423


In [41]:
trace=go.Bar(x=top10_score['batsman'],y=top10_score['batsman_runs'],

                 )
data=[trace]
layout=go.Layout()

fig=go.Figure(data=data,layout=layout)
fig.show()

###There are 2 types of Bar Graphs
1. Nested Bar Graph
2. Stacked Bar Graph
3. Overlayed Bar Graph

In [42]:
iw=top10_df.groupby(['batsman','inning'])['batsman_runs'].sum().reset_index()
mask=iw['inning']==1
mask2=iw['inning']==2
one=iw[mask]
two=iw[mask2]


one.rename(columns={'batsman_runs':'1st Innings'},inplace=True)
two.rename(columns={'batsman_runs':'2nd Innings'},inplace=True)

final=one.merge(two,on='batsman')[['batsman','1st Innings','2nd Innings']]

final

<ipython-input-42-b1bb475ddbb8>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-42-b1bb475ddbb8>:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,batsman,1st Innings,2nd Innings
0,AB de Villiers,2128,1345
1,CH Gayle,2003,1623
2,DA Warner,2118,1896
3,G Gambhir,1699,2433
4,MS Dhoni,2232,1328
5,RG Sharma,2344,1863
6,RV Uthappa,1516,2262
7,S Dhawan,2262,1299
8,SK Raina,2647,1893
9,V Kohli,2391,2027


In [43]:
trace1=go.Bar(x=final['batsman'],y=final['1st Innings'],
                  name='First innings',
                  marker={'color':'#66198c'}
                 )
trace2=go.Bar(x=final['batsman'],y=final['2nd Innings'],
                  name='First innings',
                  marker={'color':'#126e68'}
                 )
data=[trace1,trace2]
layout=go.Layout()

fig=go.Figure(data=data,layout=layout)
fig.show()

In [44]:
trace1=go.Bar(x=final['batsman'],y=final['1st Innings'],
                  name='First innings',
                  marker={'color':'#66198c'}
                 )
trace2=go.Bar(x=final['batsman'],y=final['2nd Innings'],
                  name='First innings',
                  marker={'color':'#126e68'}
                 )
data=[trace1,trace2]
layout=go.Layout(barmode='overlay')

fig=go.Figure(data=data,layout=layout)
fig.show()

In [46]:
trace1=go.Bar(x=final['batsman'],y=final['1st Innings'],
                  name='First innings',
                  marker={'color':'#66198c'}
                 )
trace2=go.Bar(x=final['batsman'],y=final['2nd Innings'],
                  name='First innings',
                  marker={'color':'#126e68'}
                 )
data=[trace1,trace2]
layout=go.Layout(barmode='stack')

fig=go.Figure(data=data,layout=layout)
fig.show()

###4. Bubble Plot
Again an extension of Scatter plot. with some additional informations

In [49]:
new_ipl=new_ipl[new_ipl['batsman_runs']==6]

six=new_ipl.groupby('batsman')['batsman_runs'].count().reset_index()

x=avg.merge(six,on='batsman')
x.head(5)

,batsman,avg,batsman_runs_x,batsman_runs_y
0,AB de Villiers,38.307692,145.129059,158
1,AC Gilchrist,27.223684,133.054662,92
2,AJ Finch,27.186441,126.299213,59
3,AM Rahane,33.593407,117.486549,60
4,AT Rayudu,27.146067,123.014257,79


In [54]:
trace=go.Scatter(x=x['avg'],y=x['batsman_runs_x'],
                 mode='markers',
                 text=x['batsman'],
                 marker={'size':x['batsman_runs_y']})
data=[trace]
layout=go.Layout(title='Bubble chart',
                 xaxis={'title':'avg'},
                 yaxis={'title':'SR'}
                 )
fig=go.Figure(data=data,layout=layout)
fig.show()

# Box plot

In [72]:
match_agg=delivery.groupby(['match_id'])['total_runs'].sum().reset_index()
season_wise=match_agg.merge(matches,left_on='match_id',right_on='id')[['match_id','total_runs','season']]
season_wise.head(5)

,match_id,total_runs,season
0,1,379,2017
1,2,371,2017
2,3,367,2017
3,4,327,2017
4,5,299,2017


In [73]:
trace=go.Box(x=season_wise['total_runs'],
                 #y=avg['batsman_runs']
                 #mode='markers',
                 #text=avg['batsman'],
                 marker={'color':'#66198c'}
             )

data=[trace]
layout=go.Layout()
fig=go.Figure(data=data,layout=layout)
fig.show()

In [78]:
trace1=go.Box(x=season_wise[season_wise['season']==2017]['total_runs'],

                 marker={'color':'#66198c'},
                 name='2017'
             )
trace2=go.Box(x=season_wise[season_wise['season']==2008]['total_runs'],
                 name='2008',
                 marker={'color':'#126e68'}
             )

data=[trace1,trace2]
layout=go.Layout()
fig=go.Figure(data=data,layout=layout)
fig.show()

### Distplot

In [82]:
import plotly.figure_factory as ff

hist_data=[avg['avg']]

group_labels=['Average']

fig=ff.create_distplot(hist_data,group_labels)

fig.show()

In [83]:
# Plot Distplot



hist_data=[avg['avg'], avg['batsman_runs']]

group_labels=['Average', 'Strike Rate']

fig=ff.create_distplot(hist_data,group_labels) #,bin_size=[10,20]

fig.show()

In [85]:
# histogram
trace=go.Histogram(x=sr['batsman_runs'],name='Strike rate variation',
              xbins={'size':2}
                 )
data=[trace]
layout=go.Layout()

fig=go.Figure(data=data,layout=layout)
fig.show()

In [86]:
# heatmap
six=delivery[delivery['batsman_runs']==6]
six=six.groupby(['batting_team','over'])['batsman_runs'].count().reset_index()

six

,batting_team,over,batsman_runs
0,Chennai Super Kings,1,9
1,Chennai Super Kings,2,21
2,Chennai Super Kings,3,49
3,Chennai Super Kings,4,45
4,Chennai Super Kings,5,53
...,...,...,...
290,Sunrisers Hyderabad,16,31
291,Sunrisers Hyderabad,17,25
292,Sunrisers Hyderabad,18,49
293,Sunrisers Hyderabad,19,58


In [87]:
# Plot Heatmap

trace=go.Heatmap(x=six['batting_team'],y=six['over'],z=six['batsman_runs'])

data=[trace]

layout=go.Layout(title='Six Heatmap')

fig=go.Figure(data=data,layout=layout)

fig.show()

In [88]:
dots=delivery[delivery['batsman_runs']==0]
dots=dots.groupby(['batting_team','over'])['batsman_runs'].count().reset_index()
dots.head(5)

,batting_team,over,batsman_runs
0,Chennai Super Kings,1,642
1,Chennai Super Kings,2,583
2,Chennai Super Kings,3,520
3,Chennai Super Kings,4,516
4,Chennai Super Kings,5,464


In [90]:
# Side by Side Heatmap



from plotly import tools

trace1=go.Heatmap(x=six['batting_team'],y=six['over'],
                 z=six['batsman_runs'].values.tolist())

trace2=go.Heatmap(x=dots['batting_team'],y=dots['over'],
                 z=dots['batsman_runs'].values.tolist())


fig=tools.make_subplots(rows=1,cols=2,subplot_titles=["6's","0's"], shared_yaxes=True)

fig.append_trace(trace1,1,1)
fig.append_trace(trace2,1,2)

fig.show()

/usr/local/lib/python3.11/dist-packages/plotly/tools.py:455: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead

